In [ ]:
# create dataframe
import pandas as pd
df = pd.DataFrame()

In [ ]:
# simulate surface current, tidal currents, wind, wind-generated waves, Coriolis Effect, water temperature, water salinity, density of sargassum, ocean oxygen level, seawater contamination for sargussum growth
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Constants
EARTH_RADIUS = 6371000  # Earth's radius in meters
OMEGA = 7.2921e-5  # Earth's angular velocity in rad/s
GRAVITY = 9.81  # Acceleration due to gravity in m/s²
WATER_DENSITY = 1025  # Seawater density in kg/m³
SARGASSUM_DENSITY = 800  # Sargassum density in kg/m³ (approximate)

# Simulation parameters
start_date = datetime(2024, 10, 1)  # Start date: October 1, 2023
end_date = datetime(2024, 10, 15)  # End date: October 15, 2023
time_step = 5 * 60  # Time step in seconds (5 minutes)
num_steps = int((end_date - start_date).total_seconds() / time_step)  # Total records

# Caribbean Sea parameters (below Jamaica)
initial_lat = 16.0  # Starting latitude (e.g., south of Jamaica)
initial_lon = -76.0  # Starting longitude (e.g., south of Jamaica)
caribbean_current_speed = 0.3  # Caribbean Current speed in m/s (approximate)
caribbean_tidal_speed = 0.05  # Tidal current speed in m/s (approximate)
caribbean_wind_speed = 6.0  # Average wind speed in m/s (trade winds)
caribbean_temperature = 29.0  # Average water temperature in °C
caribbean_salinity = 35.5  # Average salinity in PSU
caribbean_oxygen_level = 5.5  # Average oxygen level in mg/L

# Initialize arrays to store results
time = [start_date + timedelta(seconds=time_step * i) for i in range(num_steps)]
position = np.zeros((num_steps, 2))  # Latitude and longitude
current_velocity = np.zeros((num_steps, 2))  # Surface current velocity
tidal_velocity = np.zeros((num_steps, 2))  # Tidal current velocity
wind_velocity = np.zeros((num_steps, 2))  # Wind velocity
wave_velocity = np.zeros((num_steps, 2))  # Wind-generated wave velocity
coriolis_effect = np.zeros((num_steps, 2))  # Coriolis effect
temperature = np.zeros(num_steps)  # Water temperature
salinity = np.zeros(num_steps)  # Water salinity
density = np.zeros(num_steps)  # Water density
oxygen_level = np.zeros(num_steps)  # Ocean oxygen level
contamination = np.zeros(num_steps)  # Seawater contamination

# Initial conditions
position[0] = [initial_lat, initial_lon]  # Starting position
temperature[0] = caribbean_temperature  # Initial water temperature
salinity[0] = caribbean_salinity  # Initial salinity
oxygen_level[0] = caribbean_oxygen_level  # Initial oxygen level
contamination[0] = 0  # Initial contamination level

# Functions to simulate each factor
def simulate_surface_current(t):
    # Caribbean Current: Eastward and northward components
    u = caribbean_current_speed * np.sin(2 * np.pi * t / (12 * 3600))  # Eastward component in m/s
    v = caribbean_current_speed * np.cos(2 * np.pi * t / (12 * 3600))  # Northward component in m/s
    return np.array([u, v])

def simulate_tidal_current(t):
    # Tidal current with semi-diurnal period
    u = caribbean_tidal_speed * np.sin(2 * np.pi * t / (12.42 * 3600))  # Eastward component in m/s
    v = caribbean_tidal_speed * np.cos(2 * np.pi * t / (12.42 * 3600))  # Northward component in m/s
    return np.array([u, v])

def simulate_wind(t):
    # Trade winds: Eastward and northward components
    u = caribbean_wind_speed * np.sin(2 * np.pi * t / (24 * 3600))  # Eastward component in m/s
    v = caribbean_wind_speed * np.cos(2 * np.pi * t / (24 * 3600))  # Northward component in m/s
    return np.array([u, v])

def simulate_wind_generated_waves(wind_speed):
    # Wave velocity based on wind speed (empirical relationship)
    wave_speed = 0.03 * wind_speed  # Wave speed as a fraction of wind speed
    return wave_speed

def simulate_coriolis_effect(latitude, velocity):
    # Coriolis parameter: f = 2 * Omega * sin(latitude)
    f = 2 * OMEGA * np.sin(np.radians(latitude))
    # Coriolis acceleration: a = f * v (perpendicular to velocity)
    a = f * np.array([-velocity[1], velocity[0]])
    return a

def simulate_water_temperature(t):
    # Diurnal temperature variation
    return caribbean_temperature + 2 * np.sin(2 * np.pi * t / (24 * 3600))

def simulate_water_salinity(t):
    # Salinity variation due to evaporation/precipitation
    return caribbean_salinity + 0.5 * np.sin(2 * np.pi * t / (24 * 3600))

def simulate_density(temperature, salinity):
    # Simplified density calculation (UNESCO equation approximation)
    return WATER_DENSITY + 0.8 * (salinity - 35) - 0.2 * (temperature - 25)

def simulate_ocean_oxygen_level(t):
    # Oxygen level variation due to photosynthesis/respiration
    return caribbean_oxygen_level + 1 * np.sin(2 * np.pi * t / (24 * 3600))

def simulate_seawater_contamination(t):
    # Contamination level increasing over time
    return 0.1 * t / (24 * 3600)

# Main simulation loop
for i in range(1, num_steps):
    t = (time[i] - start_date).total_seconds()  # Time in seconds since start

    # Simulate factors
    current_velocity[i] = simulate_surface_current(t)
    tidal_velocity[i] = simulate_tidal_current(t)
    wind_velocity[i] = simulate_wind(t)
    wave_velocity[i] = simulate_wind_generated_waves(np.linalg.norm(wind_velocity[i]))
    coriolis_effect[i] = simulate_coriolis_effect(position[i-1, 0], current_velocity[i] + tidal_velocity[i])
    temperature[i] = simulate_water_temperature(t)
    salinity[i] = simulate_water_salinity(t)
    density[i] = simulate_density(temperature[i], salinity[i])
    oxygen_level[i] = simulate_ocean_oxygen_level(t)
    contamination[i] = simulate_seawater_contamination(t)

    # Update position (latitude and longitude)
    total_velocity = current_velocity[i] + tidal_velocity[i] + wind_velocity[i] * 0.02 + coriolis_effect[i] * time_step
    position[i] = position[i-1] + total_velocity * time_step / (EARTH_RADIUS * np.pi / 180)

data = {
    "timestamp": time,
    "Current_Velocity_U": current_velocity[:, 0],  # Eastward current velocity
    "Current_Velocity_V": current_velocity[:, 1],  # Northward current velocity
    "Tidal_Velocity_U": tidal_velocity[:, 0],  # Eastward tidal velocity
    "Tidal_Velocity_V": tidal_velocity[:, 1],  # Northward tidal velocity
    "Wind_Velocity_U": wind_velocity[:, 0],  # Eastward wind velocity
    "Wind_Velocity_V": wind_velocity[:, 1],  # Northward wind velocity
    "Wave_Velocity": np.linalg.norm(wave_velocity, axis=1),  # Wave speed
    "Coriolis_Effect_U": coriolis_effect[:, 0],  # Eastward Coriolis effect
    "Coriolis_Effect_V": coriolis_effect[:, 1],  # Northward Coriolis effect
    "Temperature": temperature,  # Water temperature
    "Salinity": salinity,  # Water salinity
    "Density": density,  # Water density
    "Oxygen_Level": oxygen_level,  # Ocean oxygen level
    "Contamination": contamination,  # Seawater contamination
}

df = pd.DataFrame(data)

In [ ]:
# create SV latitude data with sinusoidal patterns and normally distributed noise
import numpy as np
start_val = 16  # Carribean sea near Jamaica
amplitude = 0.01
num_simulations = 4033
frequency = 12
noise_std = 0.001
x = np.arange(num_simulations)
y = start_val + amplitude * np.sin(2 * np.pi * frequency * x / num_simulations)
noise = np.random.normal(0, noise_std, num_simulations)
y_noisy = y + noise
df["Latitude_SV"] = pd.DataFrame(y_noisy)

In [ ]:
# create SV longitude data with sinusoidal patterns and normally distributed noise
start_val = -76  # Carribean sea near Jamaica
amplitude = 0.01
num_simulations = 4033
frequency = 12
noise_std = 0.001
x = np.arange(num_simulations)
y = start_val + amplitude * np.sin(2 * np.pi * frequency * x / num_simulations)
noise = np.random.normal(0, noise_std, num_simulations)
y_noisy = y + noise
df["Longitude_SV"] = pd.DataFrame(y_noisy)

In [ ]:
# add normally distributed error to SV data to create satellite data
df["Latitude_Satellite"] = df["Latitude_SV"].values + np.random.normal(0, 0.0005, len(df))
df["Longitude_Satellite"] = df["Longitude_SV"].values + np.random.normal(0, 0.0005, len(df))

In [ ]:
df.to_csv("simulated_data_feb17.csv")